In [1]:
from glob import glob
import pandas as pd
import geopandas as gpd
from datetime import datetime

## Data to train the CNN

### ismn in-situ soil moisture

In [2]:
#path to single json files per station
ismn_path = r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\ground_trouth\ismn_archieve\time_series'
files = glob(ismn_path + '\*')

In [3]:
#load time series in pandas
ts_ismn_sm = [pd.read_json(x) for x in files]
# Get coords from filename
coords = [x.split('\\')[-1].split('_')[1:3] for x in files]
# Create GeoDataFrame for every time series with geometry from filename 
ts_ismn_sm_gdf = [gpd.GeoDataFrame(data = ts_ismn_sm[x], geometry=gpd.points_from_xy(x = np.repeat(coords[x][1], len(ts_ismn_sm[x])), y = np.repeat(coords[x][0], len(ts_ismn_sm[x])))) for x in range(0,len(files))]
# Concat all gdf to one geodataframe
ts_ismn = gpd.tools.util.pd.concat(ts_ismn_sm_gdf)

In [10]:
# Extract regular sampled; 1 values for 1 day at the same time s1 meassure backscatter coeffienct


### gee sentinel 1 data

In [4]:
#path to single json files per station
gee_fc_path = r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\ground_trouth\ismn_fc_gee_VV_na_clear.geojson'
ts_ismn_bc = gpd.read_file(gee_fc_path)

In [5]:
# Extract metadata from filename
ts_ismn_bc['station'] = [x[-1] for x in ts_ismn_bc.id]
ts_ismn_bc['sentinel'] = [x[2] for x in ts_ismn_bc.id]
ts_ismn_bc['date'] = [datetime.strptime(x.split('_')[4][:15], '%Y%m%dT%H%M%S') for x in ts_ismn_bc.id]
ts_ismn_bc['mode'] = [x[4:6] for x in ts_ismn_bc.id]
ts_ismn_bc['type'] = [str(x[7:11]) for x in ts_ismn_bc.id]
# only High Resolution types
ts_ismn_bc = ts_ismn_bc[ts_ismn_bc['type'] == 'GRDH']
#drop duplicates fix because when downloading vv data some sensors are at same postion but different depths
ts_ismn_bc.drop_duplicates(['geometry', 'VV'], inplace=True)

In [42]:
# Add sm from ismn data to s1 vv data
values = list()
for row in ts_ismn_bc.itertuples():
    time = row.date.round('1h')
    geometry = row.geometry
    # select all rows with same geometry (change x and y lat/lon maybe because of shaply module)
    sm = ts_ismn.cx[geometry.y:geometry.y , geometry.x:geometry.x]
    # select all rows with same time
    sm = sm[sm.index == time]
    # mean when multiple values nan when no values
    if len(sm) == 0:
        values.append(np.nan)
    else:
        values.append(np.mean(list(sm.soil_moisture)))
        
ts_ismn_bc['soil_moisture'] = values

[0.294,
 0.277,
 0.291,
 0.276,
 0.291,
 0.274,
 0.28700000000000003,
 0.272,
 0.1784,
 0.28600000000000003,
 0.272,
 0.2601,
 0.3111,
 0.321,
 0.305,
 0.25420000000000004,
 0.27690000000000003,
 0.2662,
 0.24230000000000002,
 0.298,
 0.28600000000000003,
 0.24810000000000001,
 0.296,
 0.28200000000000003,
 0.2029,
 0.293,
 0.28,
 0.37110000000000004,
 0.27690000000000003,
 0.28400000000000003,
 0.27,
 0.28350000000000003,
 0.302,
 0.295,
 0.2672,
 0.34550000000000003,
 0.3002,
 0.28600000000000003,
 0.276,
 0.39380000000000004,
 0.2677,
 0.27640000000000003,
 0.3653,
 0.32,
 0.306,
 0.2411,
 0.302,
 0.293,
 0.2207,
 0.24230000000000002,
 0.2267,
 0.28600000000000003,
 0.27,
 0.18150000000000002,
 0.17750000000000002,
 0.1733,
 0.1741,
 0.322,
 0.311,
 0.3118,
 0.294,
 0.28500000000000003,
 0.2866,
 0.37560000000000004,
 0.28200000000000003,
 0.269,
 0.42110000000000003,
 0.3225,
 0.2948,
 0.3548,
 0.294,
 0.28500000000000003,
 0.2421,
 0.28600000000000003,
 0.279,
 0.215,
 0.198,
 0.1

In [45]:
ts_ismn_bc

,id,VH,VV,angle,geometry,station,sentinel,date,mode,type,soil_moisture
34,S1A_IW_GRDH_1SDV_20150104T051921_20150104T0519...,-18.326201,-10.478161,35.847664,POINT (12.35240 43.11697),0,A,2015-01-04 05:19:21,IW,GRDH,0.294
36,S1A_IW_GRDH_1SDV_20150104T051921_20150104T0519...,-16.329298,-13.143020,35.850368,POINT (12.35196 43.11722),2,A,2015-01-04 05:19:21,IW,GRDH,0.277
38,S1A_IW_GRDH_1SDV_20150105T170525_20150105T1705...,-18.302937,-14.707238,39.530815,POINT (12.35240 43.11697),0,A,2015-01-05 17:05:25,IW,GRDH,0.291
40,S1A_IW_GRDH_1SDV_20150105T170525_20150105T1705...,-20.688267,-11.778570,39.528690,POINT (12.35196 43.11722),2,A,2015-01-05 17:05:25,IW,GRDH,0.276
42,S1A_IW_GRDH_1SDV_20150111T051106_20150111T0511...,-18.397793,-16.126218,45.326153,POINT (12.35240 43.11697),0,A,2015-01-11 05:11:06,IW,GRDH,0.291
...,...,...,...,...,...,...,...,...,...,...,...
17562,S1B_IW_GRDH_1SDV_20210925T174643_20210925T1747...,-19.836128,-12.102945,43.340008,POINT (2.95670 43.15000),0,B,2021-09-25 17:46:43,IW,GRDH,NaN
17566,S1B_IW_GRDH_1SDV_20210926T054335_20210926T0544...,-14.441960,-10.447945,45.628571,POINT (4.34483 44.22200),4,B,2021-09-26 05:43:35,IW,GRDH,NaN
17572,S1B_IW_GRDH_1SDV_20210927T173032_20210927T1730...,-16.022469,-11.260678,31.561167,POINT (4.34483 44.22200),4,B,2021-09-27 17:30:32,IW,GRDH,NaN
17578,S1B_IW_GRDH_1SDV_20210930T051112_20210930T0511...,-21.911861,-16.354178,45.354069,POINT (12.35240 43.11697),0,B,2021-09-30 05:11:12,IW,GRDH,NaN


In [ ]:
def s1_construct_gdf(p_geojson):
    """
    Arguments: Load FeatureCollection into GeoDataFrame and label data according to filename
    """
    #import modules
    import geopandas as gpd
    import numpy as np
    import pandas as pd
    from datetime import datetime
    
    gdf = gpd.read_file(p_geojson)
    gdf['patch'] = [x[-1] for x in gdf.id]
    gdf['sentinel'] = [x[2] for x in gdf.id]
    gdf['date'] = [datetime.strptime(x.split('_')[4][:15], '%Y%m%dT%H%M%S') for x in gdf.id]
    #gdf['orbit'] = ['ascending' if x.hour > 10 else 'descending' for x in gdf.date]
    gdf['identifier'] = [x[-6:-2] for x in gdf.id]
    gdf.to_file(p_geojson)
    return gdf.iloc[:,1:]